In [383]:
cols = [each['name'] for each in cat_list]
cols

['Arts & Entertainment',
 'College & University',
 'Event',
 'Food',
 'Nightlife Spot',
 'Outdoors & Recreation',
 'Professional & Other Places',
 'Residence',
 'Shop & Service',
 'Travel & Transport']

The total number of each venue category:

In [186]:
toronto_venues.sum()[1:]

Arts & Entertainment            204
College & University              9
Food                           1711
Nightlife Spot                  218
Outdoors & Recreation           340
Professional & Other Places      38
Residence                         1
Shop & Service                  551
Travel & Transport              116
dtype: object

We can check the number of venues in each Neighbourhood. Note this is currently supressed due to the length of the output.

In [213]:
# pd.DataFrame([toronto_venues.Neighbourhood, toronto_venues.sum(axis=1)],
#              index = ['Neighbourhood','Venues']).T.sort_values(
#                 by = 'Venues', ascending = False)

We can check the number of each venue category for a specific Neighbourhood (or Neighbourhoods)

In [214]:
toronto_venues.loc[toronto_venues.Neighbourhood == 'Lawrence Park']

,Neighbourhood,Arts & Entertainment,College & University,Food,Nightlife Spot,Outdoors & Recreation,Professional & Other Places,Residence,Shop & Service,Travel & Transport
18,Lawrence Park,0,2,2,0,3,0,0,1,0


As we can see here, most Neighbourhoods have 100 venues within 1 km. However, some areas have considerably less local venues.

## Top 5 venue categories - IS this still in use?

In [4]:
def getNearbyVenues(names, latitudes, longitudes, radius):
    '''This function uses the Foursquare API to search for venues in a given radius 
    around a neighbourhoods latitude and longitude.
    
    It returns a pandas dataframe with columns: Neigbourhood Name, 
    Neighbourhood Latitude, Neighbourhood Longitude, Venue Name, Venue Latitude,
    Venue Longitude and Venue Category.
    '''
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={} \
        &v={}&ll={},{}&limit={}&radius={}&sortByPopularity=1'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng,
            LIMIT,
            radius
            )
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name'],
            getParentCat(cat_list, v['venue']['categories'][0]['name'])) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',
                  'Venue Category',
                  'Parent Category']
    
    return(nearby_venues)

In [208]:
# Is this still in use?
top = toronto_venue_cats.set_index('Neighbourhood').T
top_dict = {n:top[n].sort_values(ascending=False)[0:20] for n in top}

In [211]:
# Unashamedly lazy printing:

# The ouput here is really long so is commented out temporarily

# for n in clusters:
#     print('Cluster',n,':\n')
#     for m in clusters[n]:
#         print(m,':\n')
#         for i in range(5):
#             print(top_dict[m].index[i], top_dict[m].values[i])
#         print('\n')

# As we can see above, clusters 1, 2 and 3 share very similar venue categories

In [276]:
clusters[2]

['Davisville North',
 'Dufferin, Dovercourt Village',
 'Forest Hill North & West, Forest Hill Road Park',
 'India Bazaar, The Beaches West',
 'Moore Park, Summerhill East',
 'Runnymede, Swansea',
 'Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park',
 'The Beaches']

In [275]:
[top_dict[n].values.sum()for n in clusters[2]]

[100, 68, 50, 76, 58, 73, 78, 77]

In [279]:
[top_dict[n] for n in clusters[3]]

[Food                           51
 Shop & Service                 14
 Outdoors & Recreation          12
 Nightlife Spot                  9
 Travel & Transport              7
 Arts & Entertainment            6
 Professional & Other Places     1
 College & University            0
 Name: Berczy Park, dtype: uint8,
 Food                           46
 Arts & Entertainment           15
 Shop & Service                 14
 Outdoors & Recreation           9
 Travel & Transport              9
 Nightlife Spot                  6
 Professional & Other Places     1
 College & University            0
 Name: Commerce Court, Victoria Hotel, dtype: uint8,
 Food                           46
 Arts & Entertainment           16
 Shop & Service                 13
 Travel & Transport             10
 Outdoors & Recreation           9
 Nightlife Spot                  5
 Professional & Other Places     1
 College & University            0
 Name: First Canadian Place, Underground city, dtype: uint8,
 Food       

In [248]:
top_dict['St. James Town, Cabbagetown'].values.sum()

34

In [249]:
top_dict['St. James Town, Cabbagetown']

Food                           22
Outdoors & Recreation           5
Arts & Entertainment            3
Shop & Service                  3
Nightlife Spot                  1
College & University            0
Professional & Other Places     0
Travel & Transport              0
Name: St. James Town, Cabbagetown, dtype: uint8

In [245]:
top_dict['The Beaches'].values.sum()

77

In [247]:
top_dict['The Beaches']

Food                           43
Shop & Service                 17
Outdoors & Recreation           9
Nightlife Spot                  7
Arts & Entertainment            1
College & University            0
Professional & Other Places     0
Travel & Transport              0
Name: The Beaches, dtype: uint8

# Find way to print list of cat_vals in C1 but not C2

In [31]:
list1 = [venue_cat for venue_cat in top_dic['The Beaches'].keys() if venue_cat in list(test4['C1:_venue_cat'][0:20])]
list2 = [venue_cat for venue_cat in top_dic['The Beaches'].keys() if venue_cat in list(test4['C2:_venue_cat'][0:20])]

In [32]:
diff = np.setdiff1d(list1,list2)
diff

array(['Bar', 'Breakfast Spot'], dtype='<U19')

The fact that The Beaches has these types of venues is possibly why it is in Cluster 1

In [33]:
list1 = list(test4['C1:_venue_cat'][0:20])
list2 = list(test4['C2:_venue_cat'][0:20])
list3 = list(test4['C3:_venue_cat'][0:20])
diff = np.setdiff1d(list1,list2)
diff

array(['Bar', 'Breakfast Spot', 'Indian Restaurant', 'Mexican Restaurant',
       'Vegetarian / Vegan Restaurant'], dtype='<U29')

In [34]:
diff = np.setdiff1d(list2,list1)
diff

array(['Bank', 'Gastropub', 'Gym', 'Liquor Store', 'Spa'], dtype='<U19')

In [35]:
diff = np.setdiff1d(list1,list3)
diff

array(['Bakery', 'Bar', 'Diner', 'Indian Restaurant',
       'Italian Restaurant', 'Mexican Restaurant', 'Pharmacy', 'Pub',
       'Sandwich Place', 'Sushi Restaurant',
       'Vegetarian / Vegan Restaurant'], dtype='<U29')

In [36]:
diff = np.setdiff1d(list3,list1)
diff

array(['American Restaurant', 'Beer Bar', 'Clothing Store',
       'Concert Hall', 'Deli / Bodega', 'Gastropub', 'Gym', 'Hotel',
       'Plaza', 'Steakhouse', 'Theater'], dtype='<U19')

# Note on search radius

# Different search radii section

# Let's have alook at the top venue categories for the Brockton, Parkdale Village, Exhibition Place neighbourhood

In [39]:
top_dic['Brockton, Parkdale Village, Exhibition Place']

Restaurant                0.07
Coffee Shop               0.06
Café                      0.05
Bar                       0.05
Furniture / Home Store    0.04
Bakery                    0.03
Italian Restaurant        0.02
Indian Restaurant         0.02
Pharmacy                  0.02
Art Gallery               0.02
Theater                   0.02
Sandwich Place            0.02
Park                      0.02
Thrift / Vintage Store    0.02
Tibetan Restaurant        0.02
Smoothie Shop             0.01
Grocery Store             0.01
Soccer Stadium            0.01
Falafel Restaurant        0.01
Gym                       0.01
Name: Brockton, Parkdale Village, Exhibition Place, dtype: float64